# НИС «Основы анализа данных в Python»

*Алла Тамбовцева*

## Практикум 1*. Категориальные данные и дискретизация данных в `pandas`

* Категориальные данные в `pandas`
* Дискретизация данных и функция `cut()`

### Категориальные данные в `pandas`

Если в `pandas` нас не устраивает обычный строковый тип `object` или числовые метки типа `int` для хранения данных в категориальной шкале, можно задействовать отдельный тип  – `category`. К этому типу можно привести признаки как в номинальной шкале (неупорядоченные значения), так и в порядковой (упорядоченные значения).

Создадим небольшой датафрейм на основе словаря:

In [1]:
import pandas as pd

In [2]:
data = pd.DataFrame({"id" : [1, 2, 3, 4, 5, 6, 7], 
              "пол" : ["жен", "муж", "муж", "жен", "жен", "жен", "муж"],
              "город" : ["Москва", "Пермь", "Пермь", "Ярославль", "Москва", 
                        "Москва", "Галич"],
              "v1" : [1, 0, 0, 1, 0, 1, 0], 
              "v2" : ["абсолютно не согласен (не согласна)", 
                      "абсолютно не согласен (не согласна)", 
                      "согласен (согласна)", 
                      "абсолютно согласен (абсолютно согласна)", 
                      "не согласен (не согласна)", 
                      "абсолютно согласен (абсолютно согласна)",
                      "согласен (согласна)"]})
data

,id,пол,город,v1,v2
0,1,жен,Москва,1,абсолютно не согласен (не согласна)
1,2,муж,Пермь,0,абсолютно не согласен (не согласна)
2,3,муж,Пермь,0,согласен (согласна)
3,4,жен,Ярославль,1,абсолютно согласен (абсолютно согласна)
4,5,жен,Москва,0,не согласен (не согласна)
5,6,жен,Москва,1,абсолютно согласен (абсолютно согласна)
6,7,муж,Галич,0,согласен (согласна)


Сейчас типы всех столбцов стандартные:

In [3]:
print(data.dtypes)

id        int64
пол      object
город    object
v1        int64
v2       object
dtype: object


Что стоит иметь в виду – Python умеет сравнивать текстовые строки между собой, упорядочивая их по алфавиту. То есть сейчас, если мы, например, сформулируем условие с оператором `>` для столбца `город`, ошибки не будет:

In [4]:
# все города, которые по алфавиту ниже слова `Москва`

data[data["город"] > "Москва"]

,id,пол,город,v1,v2
1,2,муж,Пермь,0,абсолютно не согласен (не согласна)
2,3,муж,Пермь,0,согласен (согласна)
3,4,жен,Ярославль,1,абсолютно согласен (абсолютно согласна)


Приведем все категориальные столбцы (кроме id) к типу `category`. Если нас устраивает сортировка по алфавиту, для приведения типа достаточно задействовать метод `.astype()`. Изменим тип столбца `город`:

In [5]:
data["город"] = data["город"].astype("category")

Внешне ничего не изменилось:

In [6]:
data

,id,пол,город,v1,v2
0,1,жен,Москва,1,абсолютно не согласен (не согласна)
1,2,муж,Пермь,0,абсолютно не согласен (не согласна)
2,3,муж,Пермь,0,согласен (согласна)
3,4,жен,Ярославль,1,абсолютно согласен (абсолютно согласна)
4,5,жен,Москва,0,не согласен (не согласна)
5,6,жен,Москва,1,абсолютно согласен (абсолютно согласна)
6,7,муж,Галич,0,согласен (согласна)


Однако тип столбца уже другой:

In [7]:
print(data.dtypes)

id          int64
пол        object
город    category
v1          int64
v2         object
dtype: object


In [8]:
print(data["город"])

0       Москва
1        Пермь
2        Пермь
3    Ярославль
4       Москва
5       Москва
6        Галич
Name: город, dtype: category
Categories (4, object): ['Галич', 'Москва', 'Пермь', 'Ярославль']


В конце выдачи со значениями столбца есть указание на то, что здесь четыре категории, а именно `['Галич', 'Москва', 'Пермь', 'Ярославль']`. Никаких содержательных изменений не произошло, города по-прежнему упорядочены по алфавиту. Однако фильтрация со знаком `>` уже не сработает. По умолчанию, если мы нигде не фиксировали, что значения упорядочены, Python понимает, что значения типа `category` сравнивать друг с другом через `>` и `<` уже нельзя:

In [ ]:
# раскомментируйте строку ниже,
# получите ошибку – Unordered Categoricals

# data[data["город"] > "Москва"]

То же можно проделать и с числовыми метками, если есть необходимость. Например, бинарный столбец `v1` тоже можно сделать категориальным:

In [9]:
data["v1"] = data["v1"].astype("category")

In [10]:
data["v1"]

0    1
1    0
2    0
3    1
4    0
5    1
6    0
Name: v1, dtype: category
Categories (2, int64): [0, 1]

Теперь этот признак стал категориальным, однако сами значения не превратились в текст. В выдаче в конце сказано, что здесь две категории, однако их значения – целочисленные (`int64`). Получается, при выборе строк мы по-прежнему сможем использовать целые числа:

In [11]:
data[data["v1"] == 0]

,id,пол,город,v1,v2
1,2,муж,Пермь,0,абсолютно не согласен (не согласна)
2,3,муж,Пермь,0,согласен (согласна)
4,5,жен,Москва,0,не согласен (не согласна)
6,7,муж,Галич,0,согласен (согласна)


In [12]:
data[data["v1"] != 0]

,id,пол,город,v1,v2
0,1,жен,Москва,1,абсолютно не согласен (не согласна)
3,4,жен,Ярославль,1,абсолютно согласен (абсолютно согласна)
5,6,жен,Москва,1,абсолютно согласен (абсолютно согласна)


А вот использовать операторы, кроме `=` и `!=`, уже не сможем:

In [ ]:
# раскомментируйте строку ниже,
# получите ошибку – Unordered Categoricals

# data[data["v1"] > 0]

Если категории нужно упорядочить, и порядок отличается от алфавитного (данные в порядковой шкале или собираемся строить модели с определенной базовой категорией), метода `.astype()` не хватит, понадобится функция `Categorical()`. Внутри этой функции помимо самого столбца с данными можно зафиксировать порядок категорий и «включить» упорядочение, если требуется. 

Проделаем разумное упорядочивание для столбца `v2`:

In [13]:
data["v2"] = pd.Categorical(data["v2"], 
               categories = ["абсолютно не согласен (не согласна)", 
                            "не согласен (не согласна)",
                            "согласен (согласна)",
                            "абсолютно согласен (абсолютно согласна)"], 
              ordered = True)

Проверим тип:

In [15]:
print(data["v2"])

0        абсолютно не согласен (не согласна)
1        абсолютно не согласен (не согласна)
2                        согласен (согласна)
3    абсолютно согласен (абсолютно согласна)
4                  не согласен (не согласна)
5    абсолютно согласен (абсолютно согласна)
6                        согласен (согласна)
Name: v2, dtype: category
Categories (4, object): ['абсолютно не согласен (не согласна)' < 'не согласен (не согласна)' < 'согласен (согласна)' < 'абсолютно согласен (абсолютно согласна)']


В выдаче в перечне с уникальными значениями в конце добавились знаки `<`. А значит, теперь для выбора строк можно использовать операторы `<` и `>` (и нестрогие аналоги). Выберем всех согласных (и абсолютно согласных) с некоторым утверждением:

In [16]:
# алфавит ни на что не влияет
# выбираем все, что по нашему порядку не меньше

data[data["v2"] >= "согласен (согласна)"]

,id,пол,город,v1,v2
2,3,муж,Пермь,0,согласен (согласна)
3,4,жен,Ярославль,1,абсолютно согласен (абсолютно согласна)
5,6,жен,Москва,1,абсолютно согласен (абсолютно согласна)
6,7,муж,Галич,0,согласен (согласна)


Два нюанса:

* Если в `Categorical()` не добавлять аргумент `categories`, но оставить `ordered = True`, мы получим порядковый признак, но Python сам упорядочит уникальные значения этого столбца по алфавиту. То есть, мы получим то же, что и через `.astype('category')`, только будет разрешены сравнения с `>`, `>=`, `<`, `<=`.

* Если в `categories` мы потеряем какую-то категорию, соответствующие ей значения в столбце будут заменены на пропуски `NaN`. Теоретически, это можно использовать намеренно – если нас интересуют только определенные ответы, можно перечислить их в `categories`, остальные заменятся на `NaN`, а потом при выкидывании мы их удалим.

В заключение посмотрим, что происходит при описании категориальных столбцов:

In [17]:
print(data.dtypes)

id          int64
пол        object
город    category
v1       category
v2       category
dtype: object


In [18]:
print(data["город"].describe())

count          7
unique         4
top       Москва
freq           3
Name: город, dtype: object


In [19]:
print(data["v1"].describe())

count     7
unique    2
top       0
freq      4
Name: v1, dtype: int64


In [20]:
print(data["v2"].describe())

count                                       7
unique                                      4
top       абсолютно не согласен (не согласна)
freq                                        2
Name: v2, dtype: object


У столбцов, которые изначально были текстовыми, ничего нового нет – для типа `category`, как и для `object`, выводится число заполненных ячеек `count`, число уникальных значений `unique`, мода `top` и частота, соответствующая моде `freq`. А вот со столбцом `v1` интересно. Сам он имеет тип `category` (см выше в `.dtypes`), но его значения имеют тип `int64`. Поэтому значения в столбце по-прежнему целочисленные, но описывается он не как числовой (минимум, максимум, среднее и других стандартные описательные статистик), а как текстовый.

### Дискретизация данных и функция `cut()`

Загрузим данные из файла к основному практикуму `disney-clean.csv` по ссылке и при загрузке выберем только те столбцы, которые нам будут нужны для работы:

In [21]:
url = "https://raw.githubusercontent.com/allatambov/PyDat25/refs/heads/main/disney_clean.csv"

# если заранее знаем названия нужных столбцов,
# можем указать их в usecols

disney = pd.read_csv(url, usecols = ["Title", "Release year", "IMDB"])
disney

,Title,IMDB,Release year
0,Academy Award Review of,7.2,1937.0
1,Snow White and the Seven Dwarfs,7.6,1937.0
2,Pinocchio,7.4,1940.0
3,Fantasia,7.8,1940.0
4,The Reluctant Dragon,6.9,1941.0
...,...,...,...
427,Soul,7.0,2020.0
428,Raya and the Last Dragon,NaN,2021.0
429,Cruella,NaN,2021.0
430,Jungle Cruise,NaN,2021.0


Что такое дискретизация данных? Разбиение непрерывного признака на интервалы, которые можно закодировать конечным набором значений. Превращение количественного непрерывного признака в количественный дискретный признак по заданному правилу. Однако далее мы будем рассматривать не столько дискретизацию в чистом виде, сколько вообще примеры разбиения числового показателя на категории, которые можно упорядочить.

В датафрейме `disney` формально столбец `IMDB` можно считать непрерывным признаком, поскольку это рейтинг после усреднения, и без округления теоретически он может быть со сколько угодно знаками после точки. Посмотрим на его описательные статистики – пригодятся чуть позже:

In [22]:
disney["IMDB"].describe()

count    416.000000
mean       6.545433
std        0.960937
min        1.500000
25%        6.000000
50%        6.600000
75%        7.200000
max        8.700000
Name: IMDB, dtype: float64

Если мы готовы поступиться точностью (иногда это тоже требуется), нам нужно будет разбить диапазон всех значений рейтинга на интервалы. Сделать это можно с помощью функции `cut()`.

На вход этой функции нужно передать числовой столбец и правило для разбиения – количество равных по размеру интервалов или сами границы интервалов в явном виде. Начнем с простого: попробуем разбить все значения `IMDB` на 5 групп:

In [23]:
pd.cut(disney["IMDB"], bins = 5)

0      (5.82, 7.26]
1       (7.26, 8.7]
2       (7.26, 8.7]
3       (7.26, 8.7]
4      (5.82, 7.26]
           ...     
427    (5.82, 7.26]
428             NaN
429             NaN
430             NaN
431     (7.26, 8.7]
Name: IMDB, Length: 432, dtype: category
Categories (5, interval[float64, right]): [(1.493, 2.94] < (2.94, 4.38] < (4.38, 5.82] < (5.82, 7.26] < (7.26, 8.7]]

Функция сделала следующее:

1. Вычислила минимум (1.5) и максимум (8.7) по столбцу, разбила интервал [1.5, 8.7] на 5 равных отрезков:

        (1.493, 2.94], (2.94, 4.38], (4.38, 5.82], (5.82, 7.26], (7.26, 8.7]

   Длина каждого отрезка 1.44 (минимум на самом деле 1.5, а не 1.493, поэтому все ок). Отрезки по умолчанию открыты слева, то есть левая граница в сам отрезок не входит. Так, значение 4.38 будет принадлежать второму отрезку, а не третьему. Такое правило задания границ отрезков объясняет, почему первое значение в первом отрезке не 1.5 – чтобы не потерять минимум, взяв отрезок `(1.5, 2.94]`, Python немного отступает от минимального значения назад.

2. Вернула новый столбец типа `category` из упорядоченных значений 

        (1.493, 2.94] < (2.94, 4.38] < (4.38, 5.82] < (5.82, 7.26] < (7.26, 8.7].
        
Если мы добавим такой столбец в датафрейм, будет не очень удобно:

In [24]:
disney["IMDB Groups"] = pd.cut(disney["IMDB"], bins = 5)
disney.head()

,Title,IMDB,Release year,IMDB Groups
0,Academy Award Review of,7.2,1937.0,"(5.82, 7.26]"
1,Snow White and the Seven Dwarfs,7.6,1937.0,"(7.26, 8.7]"
2,Pinocchio,7.4,1940.0,"(7.26, 8.7]"
3,Fantasia,7.8,1940.0,"(7.26, 8.7]"
4,The Reluctant Dragon,6.9,1941.0,"(5.82, 7.26]"


Поэтому добавим числовые метки для полученных отрезков (порядковая шкала, раз отрезки идут друг за другом на числовой прямой) внутри `cut()`:

In [25]:
# так лучше

disney["IMDB Groups"] = pd.cut(disney["IMDB"], bins = 5, 
                              labels = [1, 2, 3, 4, 5])
disney.head()

,Title,IMDB,Release year,IMDB Groups
0,Academy Award Review of,7.2,1937.0,4
1,Snow White and the Seven Dwarfs,7.6,1937.0,5
2,Pinocchio,7.4,1940.0,5
3,Fantasia,7.8,1940.0,5
4,The Reluctant Dragon,6.9,1941.0,4


Посмотрим, сколько значений в каждом интервале (в каждой категории):

In [26]:
# много значений в категории 4 и 5,
# это высокие значения рейтинга

disney["IMDB Groups"].value_counts()

4    240
5     91
3     74
2      9
1      2
Name: IMDB Groups, dtype: int64

Если нас не устраивает дробный шаг или интервалы одинаковой длины, мы можем самостоятельно зафиксировать границы отрезков:

In [27]:
# от 1 до 9 с шагом 2

var01 = pd.cut(disney["IMDB"], bins = [1, 3, 5, 7, 9])
var01.value_counts()

(5, 7]    256
(7, 9]    134
(3, 5]     24
(1, 3]      2
Name: IMDB, dtype: int64

In [28]:
# отрезки (1, 5], (5, 7] и так далее

var02 = pd.cut(disney["IMDB"], bins = [1, 5, 7, 8, 9])
var02.value_counts()

(5, 7]    256
(7, 8]    122
(1, 5]     26
(8, 9]     12
Name: IMDB, dtype: int64

Аналогичным образом попробуем поработать с годом выхода мультфильма – разобьем годы на десятилетия. Попутно решим любопытную задачу – как автоматически определить, с какого десятилетия начать? Определим год выхода самого раннего мультфильма:

In [29]:
disney["Release year"].min()

1937.0

Как отсюда получить десятилетие в виде числа `1930`? Вычтем из года остаток от деления на 10:

In [30]:
min_ = disney["Release year"].min()
start = min_  - min_ % 10
print(start)

1930.0


Похожим образом определим, на каком десятилетии нужно остановиться:

In [31]:
disney["Release year"].max()

2021.0

In [32]:
max_ = disney["Release year"].max()

# + вперед уходим на 10 лет с запасом
end = max_  - max_ % 10 + 10
print(end)

2030.0


Чтобы не перечислять годы вручную, воспользуемся функцией `np.arange()` для создания границ интервалов. Функция `range()` не совсем подойдет – год, конечно, по идее целочисленный, но из-за наличия пропусков тип столбца `float`, поэтому нужна функция, умеющая работать с дробными числами:

In [33]:
import numpy as np

In [34]:
pd.cut(disney["Release year"], 
       bins = np.arange(start, end + 10, 10))

0      (1930.0, 1940.0]
1      (1930.0, 1940.0]
2      (1930.0, 1940.0]
3      (1930.0, 1940.0]
4      (1940.0, 1950.0]
             ...       
427    (2010.0, 2020.0]
428    (2020.0, 2030.0]
429    (2020.0, 2030.0]
430    (2020.0, 2030.0]
431    (2020.0, 2030.0]
Name: Release year, Length: 432, dtype: category
Categories (10, interval[float64, right]): [(1930.0, 1940.0] < (1940.0, 1950.0] < (1950.0, 1960.0] < (1960.0, 1970.0] ... (1990.0, 2000.0] < (2000.0, 2010.0] < (2010.0, 2020.0] < (2020.0, 2030.0]]

Добавим вразумительные метки, они могут быть и текстовыми, упорядочение сохранится:

In [35]:
labs = [str(i) + "s" for i in range(1930, 2030, 10)]
print(labs)

['1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']


In [36]:
disney["Release Decade"] = pd.cut(disney["Release year"], 
       bins = np.arange(start, end + 10, 10),
       labels = labs)

disney

,Title,IMDB,Release year,IMDB Groups,Release Decade
0,Academy Award Review of,7.2,1937.0,4,1930s
1,Snow White and the Seven Dwarfs,7.6,1937.0,5,1930s
2,Pinocchio,7.4,1940.0,5,1930s
3,Fantasia,7.8,1940.0,5,1930s
4,The Reluctant Dragon,6.9,1941.0,4,1940s
...,...,...,...,...,...
427,Soul,7.0,2020.0,4,2010s
428,Raya and the Last Dragon,NaN,2021.0,NaN,2020s
429,Cruella,NaN,2021.0,NaN,2020s
430,Jungle Cruise,NaN,2021.0,NaN,2020s


P.S. Можете также ознакомиться с [функцией](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html) `qcut()`, которая использует статистический подход для разбиения на группы – по квантилям.